# MoCap Arena

---

In [66]:
# Importing modules...
import numpy as np
import scipy as sp
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.vision.camera import Camera
from modules.vision.multiple_view import MultipleView
from modules.vision.epipolar_geometry import build_essential_matrix, build_fundamental_matrix, order_centroids

from modules.plot.graph import Graph

# Instanciating Multiple View Object

To generate the data regarding the mathematical model of multiple vision triangulation in *CoppeliaSim*, it will be instanciated a Multiple View Object, composed of a set of Camera Obejcts that matches the Vision Sensors' parameters. 

---

In [67]:
n_clients = 4
cameras = [] # Camera objects list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    object_matrix = R @ base_matrix

    cameras.append(Camera(# Intrinsic Parameters
                          resolution=(720,720), 
                          fov_degrees=60.0,     
          
                          # Extrinsic Parameters
                          object_matrix=object_matrix,
          
                          # Rational Lens Distortion Model
                          # distortion_model='rational',
                          # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]), 
                          
                          # Fisheye Lens Distortion Model
                          distortion_model='fisheye',
                          distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
          
                          # Image Noise Model
                          snr_dB=13
                          ))
    
multiple_view = MultipleView(cameras)

# Creating the Server's UDP Socket

---

In [69]:
print(f'[SERVER] Creating socket...')

# Try to create server socket
try: 
    server_socket = socket.socket(socket.AF_INET,    # Internet
                                  socket.SOCK_DGRAM) # UDP
    print(f'[SERVER] Socket successfully created')
    
except socket.error as err: 
    print(f'[SERVER] Socket creation failed with error {err}\n')
    print(f'[SERVER] Quitting code...')
    exit()

server_ip = '127.0.0.1' # Server IP
server_port = 8888      # Server Port
server_address = (server_ip, server_port) 

server_socket.bind(server_address)
print(f'[SERVER] Bound to port {server_port}')

buffer_size = 1024 # Size of the messages in bytes

[SERVER] Creating socket...
[SERVER] Socket successfully created
[SERVER] Bound to port 8888


# Declaring the Controller's UDP Socket

---

In [70]:
# Controller socket info
controller_ip = '127.0.0.1' # Controller IP
controller_port = 7777      # Controller Port
controller_address = (controller_ip, controller_port)

# Sending Capture Info

---

In [71]:
simulation_time = 10 # In seconds

# Send capture info to the controller
message = f'{n_clients}, {simulation_time}'

message_bytes = message.encode()

server_socket.sendto(message_bytes, controller_address)
print(f'[SERVER] Capture info sent')

# Wait for controller to setup the info
server_socket.recvfrom(buffer_size)
print(f'[SERVER] Clients created')

[SERVER] Capture info sent
[SERVER] Clients created


# Sending Vision Sensor Parameters

---

In [72]:
def coppelia_vision_sensor_wrapper(camera):
    parameter_array = np.array([# Options
                                2+4, # Bit 1 set: Perspective Mode
                                     # Bit 2 set: Invisible Viewing Frustum 
                                                                     
                                # Integer parameters
                                camera.resolution[0], 
                                camera.resolution[1],
                                0, # Reserved
                                0, # Reserved

                                # Float parameters
                                0.01, # Near clipping plane in meters
                                10, # Far clipping plane in meters
                                camera.fov_radians, # FOV view angle in radians
                                0.1, # Sensor X size
                                0.0, # Reserved
                                0.0, # Reserved
                                0.0, # Null pixel red-value
                                0.0, # Null pixel green-value
                                0.0, # Null pixel blue-value
                                0.0, # Reserved
                                0.0, # Reserved
                                ], 
                               dtype=np.float64)

    transformation_array = camera.coppeliasim_object_matrix.ravel()
    
    buffer = np.concatenate((parameter_array, transformation_array)).tobytes()

    return buffer

print(f'[SERVER] Sending Vision Sensors\' parameters')

for ID, C in enumerate(multiple_view.cameras):
    # Wrap vision sensor parameters
    buffer = coppelia_vision_sensor_wrapper(C)

    server_socket.sendto(buffer, controller_address)

    # Wait for controller to setup the info
    server_socket.recvfrom(buffer_size)
    print(f'\tVision Sensor {ID} created')

[SERVER] Sending Vision Sensors' parameters
	Vision Sensor 0 created
	Vision Sensor 1 created
	Vision Sensor 2 created
	Vision Sensor 3 created


# Sending trigger

---

In [73]:
# Wait for controller to setup the scene
server_socket.recvfrom(buffer_size)
print(f'[SERVER] Scene ready')

# Send trigger
server_socket.sendto(''.encode(), controller_address)
print(f'[SERVER] Trigger sent!')

[SERVER] Scene ready
[SERVER] Trigger sent!


# Handshaking Clients

---

In [74]:
client_addresses = np.empty(n_clients)

print(f'[SERVER] Waiting for clients...')

# Address lookup 
while len(client_addresses.keys()) < n_clients: # Until all clients are identified
    message_bytes, address = server_socket.recvfrom(buffer_size)

    try:
        ID = int(message_bytes.decode()) # Decode message

    except: # Invalid message for decoding
        continue # Look for another message

    client_addresses[address] = ID

    print(f'\tClient {ID} connected')

print(f'[SERVER] All clients connected!')

[SERVER] Waiting for clients...
	Client 0 connected
	Client 1 connected
	Client 2 connected
	Client 3 connected
[SERVER] All clients connected!


# Running Simulation

---

In [75]:
timeout = 5 # In seconds
server_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Triangulation pair
pair = (0, 1)
reference, auxiliary = pair

# Received messages
received_messages = [[] for _ in range(n_clients)]

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server_socket.recvfrom(buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        break # Close loop due to timeout

    print(f'> Received message from Client {client_addresses[address]} ({address[0]}, {address[1]}):')

    # Decode message 
    message = np.frombuffer(message_bytes, dtype=np.float64)

    PTS = message[-1] # Last element of the message contains PTS
    received_messages[client_addresses[address]].append(PTS)

    # If no blobs were detected then image is invalid
    if len(message[:-1]) <= 1: # Message has no blobs, only PTS
        print(f'\tNo blobs were detected - {PTS :.3f} s')
        continue # Jump wait for next message

    detected_blobs = message[:-1].reshape(-1, 2) # All but last element are the blob coordinates

    print(f'\tDetected Blobs - {PTS :.3f} s')
    print('\t' + str(detected_blobs).replace('\n', '\n\t'))

    

    '''# If no blobs were detected then images are invalid
    if any(blob is None for blob in distorted_blobs):
        continue # Jump to next iteration
    
    # If blob count is different between each image
    if distorted_blobs[reference].shape != distorted_blobs[auxiliary].shape:
        continue # Jump to next iteration
    
    # Undistort blobs
    undistorted_blobs = [camera[ID].undistort_points(distorted_blobs[ID].T.reshape(1, -1, 2).astype(np.float32), 
                                                     camera[ID].intrinsic_matrix, 
                                                     camera[ID].distortion_coefficients,
                                                     np.array([]),
                                                     camera[ID].intrinsic_matrix).reshape(-1,2).T for ID in pair]

    # Order blobs
    epilines_auxiliary = cv2.computeCorrespondEpilines(points=undistorted_blobs[reference].T, 
                                                       whichImage=1, 
                                                       F=fundamental_matrix[reference][auxiliary]).reshape(-1,3)
    
    undistorted_blobs[auxiliary] = order_centroids(undistorted_blobs[auxiliary], epilines_auxiliary)

    # Triangulate markers
    triangulated_positions = cv2.triangulatePoints(camera[reference].projection_matrix.astype(float), 
                                                   camera[auxiliary].projection_matrix.astype(float), 
                                                   undistorted_blobs[reference].astype(float), 
                                                   undistorted_blobs[auxiliary].astype(float))
    
    # Normalize homogeneous coordinates and save
    triangulated_positions = (triangulated_positions / triangulated_positions[-1])[:-1, :]
    estimated_marker_positions[t] = triangulated_positions

    # Position triangulation tag
    sim.setObjectPosition(tag_handle, triangulated_positions.reshape(-1, 3)[0].tolist())

    # Get ground truth marker position for comparison
    true_marker_position = np.array(sim.getObjectPosition(sim.getObject('/Marker'))).reshape(3,-1)
    true_marker_positions[t] = true_marker_position'''


> Server timeout set to 5 seconds

> Received message from Client 0 (127.0.0.1, 53010):
	Detected Blobs - 0.100 s
	[[359.30000019 246.30000019]]
> Received message from Client 1 (127.0.0.1, 53011):
	Detected Blobs - 0.100 s
	[[359.30000019 246.30000019]]
> Received message from Client 2 (127.0.0.1, 58046):
	Detected Blobs - 0.100 s
	[[359.30000019 246.30000019]]
> Received message from Client 3 (127.0.0.1, 58047):
	Detected Blobs - 0.100 s
	[[359.30000019 246.30000019]]
> Received message from Client 1 (127.0.0.1, 53011):
	Detected Blobs - 0.150 s
	[[359.30000019 243.30000019]]
> Received message from Client 0 (127.0.0.1, 53010):
	Detected Blobs - 0.200 s
	[[359.30000019 241.30000019]]
> Received message from Client 2 (127.0.0.1, 58046):
	Detected Blobs - 0.200 s
	[[359.30000019 240.30000019]]
> Received message from Client 0 (127.0.0.1, 53010):
	Detected Blobs - 0.250 s
	[[359.14938927 238.61400747]]
> Received message from Client 1 (127.0.0.1, 53011):
	Detected Blobs - 0.250 s
	[[359

In [76]:
# Create graph
messages_plot = Graph(title='Received Messages - Asynchronous',
                      axis_title=('PTS', 'Clients'))

# Adding received client messages
for ID in range(n_clients):
    client_messages = np.array(received_messages[ID])
    messages_per_PTS = np.vstack((client_messages, np.full(client_messages.shape, ID)))
    messages_plot.add_points(messages_per_PTS, f'Client {ID}')

# Plot image
messages_plot.figure.show(renderer='notebook_connected')

# Analysing Precision and Accuracy 

Since the ground truth data of the marker's position is available, it's possible to evaluate the Precision and Accuracy of the system for the capture of a single stationary marker on the scene.

Calculating the average difference between the position of the marker it's ground truth will determine the **Precision**. Calculating the standard deviation of the marker's position will determine how accurate the system is by it's **Jitter**.

---

In [77]:
'''static_capture = False # Flag if the marker is moving or not in the scene 

# Gathering Data
all_marker_positions = None
triangulation_errors = None
for frame, pts in enumerate(estimated_marker_positions.keys()):
    if all_marker_positions is None:
        all_marker_positions = estimated_marker_positions[pts]
    else:
        all_marker_positions = np.hstack((all_marker_positions, estimated_marker_positions[pts]))

    if triangulation_errors is None:
        triangulation_errors = np.linalg.norm(estimated_marker_positions[pts] - true_marker_positions[pts])
    else:
        triangulation_errors = np.hstack((triangulation_errors, np.linalg.norm(estimated_marker_positions[pts] - true_marker_positions[pts])))

# For static single marker capture
if static_capture == True:
    true_position = np.ravel(list(true_marker_positions.values())[0]) # Ground truth
    average_position = np.mean(all_marker_positions, axis=1) # For Precision
    std_dev_position = np.std(all_marker_positions, axis=1)  # For Accuracy 

    print('Ground Truth Position:', true_position)
    print('Average Position:', average_position) 
    print(f'Average Reconstruction Error: {np.mean(triangulation_errors) * 1e3 :.2f} mm') # Precision
    print(f'Standard Deviation: {np.linalg.norm(std_dev_position) * 1e3 :.2f} mm')        # Accuracy

# For moving single marker capture
else:
    print(f'Average Reconstruction Error: {np.mean(triangulation_errors) * 1e3 :.2f} mm') # Precision'''

"static_capture = False # Flag if the marker is moving or not in the scene \n\n# Gathering Data\nall_marker_positions = None\ntriangulation_errors = None\nfor frame, pts in enumerate(estimated_marker_positions.keys()):\n    if all_marker_positions is None:\n        all_marker_positions = estimated_marker_positions[pts]\n    else:\n        all_marker_positions = np.hstack((all_marker_positions, estimated_marker_positions[pts]))\n\n    if triangulation_errors is None:\n        triangulation_errors = np.linalg.norm(estimated_marker_positions[pts] - true_marker_positions[pts])\n    else:\n        triangulation_errors = np.hstack((triangulation_errors, np.linalg.norm(estimated_marker_positions[pts] - true_marker_positions[pts])))\n\n# For static single marker capture\nif static_capture == True:\n    true_position = np.ravel(list(true_marker_positions.values())[0]) # Ground truth\n    average_position = np.mean(all_marker_positions, axis=1) # For Precision\n    std_dev_position = np.std(all_

# Playing Back the Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

---

In [78]:
'''# Playback a camera image feed in real time
ID = 0 # Camera ID to be watched
timestep = int(1e3 * simulation_time/len(images.keys())) # In milliseconds
for frame, pts in enumerate(images.keys()):
    cv2.imshow(f'Camera {ID}', images[pts][ID])
    cv2.waitKey(timestep)

# Closing all open windows 
cv2.destroyAllWindows()'''

"# Playback a camera image feed in real time\nID = 0 # Camera ID to be watched\ntimestep = int(1e3 * simulation_time/len(images.keys())) # In milliseconds\nfor frame, pts in enumerate(images.keys()):\n    cv2.imshow(f'Camera {ID}', images[pts][ID])\n    cv2.waitKey(timestep)\n\n# Closing all open windows \ncv2.destroyAllWindows()"